In [95]:
# dependencies
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import numpy as np
import pandas as pd
import addfips
import random
plotly.tools.set_credentials_file(username='michaelgarza', api_key='Dqqcuh2voaSqHOqQFP5W')

In [103]:
# reading in csv file that contains county information to create the choropleth
df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
df_aqi = pd.read_csv("Resources/annual_aqi_by_county_2018.csv")
df_aqi

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,Alabama,DeKalb,2018,58,58,0,0,0,0,0,44,37,31,0,0,58,0,0,0
1,Alabama,Etowah,2018,59,27,31,0,1,0,0,153,62,52,0,0,0,0,59,0
2,Alabama,Jefferson,2018,59,38,21,0,0,0,0,72,57,47,0,1,6,7,44,1
3,Alabama,Mobile,2018,59,55,4,0,0,0,0,53,38,17,0,0,0,20,39,0
4,Alabama,Morgan,2018,59,54,5,0,0,0,0,56,50,29,0,0,0,0,59,0
5,Alabama,Russell,2018,59,35,24,0,0,0,0,84,63,45,0,0,0,0,59,0
6,Alabama,Shelby,2018,58,57,0,1,0,0,0,124,9,0,0,0,0,58,0,0
7,Alabama,Sumter,2018,59,59,0,0,0,0,0,45,36,19,0,0,0,9,50,0
8,Alabama,Tuscaloosa,2018,54,43,11,0,0,0,0,62,54,38,0,0,0,0,54,0
9,Alaska,Denali,2018,90,90,0,0,0,0,0,49,44,39,0,0,90,0,0,0


In [115]:
df_sample=pd.concat([df_sample, df_aqi],axis=1)

df_sample

,LAUS Code,State FIPS Code,County FIPS Code,County Name/State Abbreviation,Year,Labor Force,Employed,Unemployed,Unemployment Rate (%),State,...,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,CN0100100000000,1,1,"Autauga County, AL",2016,"25,649","24,297","1,352",5.3,Alabama,...,0.0,44.0,37.0,31.0,0.0,0.0,58.0,0.0,0.0,0.0
1,CN0100300000000,1,3,"Baldwin County, AL",2016,"89,931","85,061","4,870",5.4,Alabama,...,0.0,153.0,62.0,52.0,0.0,0.0,0.0,0.0,59.0,0.0
2,CN0100500000000,1,5,"Barbour County, AL",2016,"8,302","7,584",718,8.6,Alabama,...,0.0,72.0,57.0,47.0,0.0,1.0,6.0,7.0,44.0,1.0
3,CN0100700000000,1,7,"Bibb County, AL",2016,"8,573","8,004",569,6.6,Alabama,...,0.0,53.0,38.0,17.0,0.0,0.0,0.0,20.0,39.0,0.0
4,CN0100900000000,1,9,"Blount County, AL",2016,"24,525","23,171","1,354",5.5,Alabama,...,0.0,56.0,50.0,29.0,0.0,0.0,0.0,0.0,59.0,0.0
5,CN0101100000000,1,11,"Bullock County, AL",2016,"4,811","4,466",345,7.2,Alabama,...,0.0,84.0,63.0,45.0,0.0,0.0,0.0,0.0,59.0,0.0
6,CN0101300000000,1,13,"Butler County, AL",2016,"9,132","8,481",651,7.1,Alabama,...,0.0,124.0,9.0,0.0,0.0,0.0,0.0,58.0,0.0,0.0
7,CN0101500000000,1,15,"Calhoun County, AL",2016,"45,794","42,727","3,067",6.7,Alabama,...,0.0,45.0,36.0,19.0,0.0,0.0,0.0,9.0,50.0,0.0
8,CN0101700000000,1,17,"Chambers County, AL",2016,"15,134","14,295",839,5.5,Alabama,...,0.0,62.0,54.0,38.0,0.0,0.0,0.0,0.0,54.0,0.0
9,CN0101900000000,1,19,"Cherokee County, AL",2016,"11,163","10,581",582,5.2,Alaska,...,0.0,49.0,44.0,39.0,0.0,0.0,90.0,0.0,0.0,0.0


In [120]:
df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']

colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]
endpts = list(np.linspace(1, 12, len(colorscale) - 1))
fips = df_sample['FIPS'].tolist()
values = df_sample['Median AQI'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Median AQI',
    legend_title='% Air Quality Index'
)
py.iplot(fig, filename='choropleth_full_usa')

/anaconda3/envs/project_1/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False


/anaconda3/envs/project_1/lib/python3.6/site-packages/plotly/plotly/plotly.py:224: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~michaelgarza/0 or inside your plot.ly account where it is named 'choropleth_full_usa'


/anaconda3/envs/project_1/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [117]:
# the following block of code is to create the choropleth
